In [ ]:
# --- Preprocess (order matters, per 6.4212 notes) ---
for each camera c:
    Pc = CROP_TO_AABB(P_c, bin_box)
    Nc = ESTIMATE_NORMALS(Pc, use_viewpoint=T_c.origin)  # flip normals toward camera
WorldPts = CONCAT( { TRANSFORM(T_c, Pc) } )
WorldNrm = CONCAT( { ROTATE(T_c, Nc) } )
(P, N) = VOXEL_DOWNSAMPLE(WorldPts, WorldNrm, voxel=v)   # one point+normal per voxel

φ = atan(μ)                # friction half-angle
ε_col = cos(5°)            # colinearity tightness (tune 2–10°)
G = ∅

# --- Sampling loop ---
for k in 1..K:
    i = SAMPLE_INDEX(P, weight=SALIENCY(P,N))             # bias to edges/curvature
    (p_i, n_i) = (P[i], N[i])

    Cands = NEIGHBORS_WITHIN(P, p_i, r_max=w_max)
    Cands = [ j in Cands |
              w = ||P[j]-p_i|| ∈ [w_min, w_max] AND
              DOT(N[j], -n_i) ≥ ε_col ]                   # approx opposite normals

    if Cands is empty: continue
    j = RANDOM_CHOICE(Cands)
    (p_j, n_j) = (P[j], N[j])
    axis = UNIT(p_j - p_i)
    width = NORM(p_j - p_i)

    # --- Colinear antipodal tests ---
    # 1) Normals oppose each other
    if DOT(n_i, -n_j) < ε_col: continue
    # 2) Axis colinear with both normals (both contacts approx "head-on")
    if |DOT(n_i,  axis)| < ε_col: continue
    if |DOT(n_j, -axis)| < ε_col: continue
    # 3) Friction cones admit closing forces (antipodal under μ)
    if ANGLE(n_i,  axis) > φ: continue
    if ANGLE(n_j, -axis) > φ: continue

    # Optional: collision / reachability
    if FINGER_SWEPT_VOLUME_COLLIDES(p_i, p_j, axis): continue
    if not IK_REACHABLE(p_i, p_j): continue

    # --- Score (simple, class-friendly) ---
    s_col = 0.5*(|DOT(n_i,axis)| + |DOT(n_j,-axis)|)      # colinearity
    s_op  = DOT(n_i, -n_j)                                 # opposition
    s_fc  = min( φ - ANGLE(n_i,axis), φ - ANGLE(n_j,-axis) )  # cone margin
    s_w   = 1 - |width - mid(w_min,w_max)| / span(w_min,w_max)
    score = w1*s_fc + w2*s_col + w3*s_op + w4*s_w

    G.add( (p_i, p_j, axis, width, score) )

return TOP_K(G, k_best)
